In [1]:
!pip install chromadb
!pip install sentence_transformers
!pip install PyPDF2
!pip install python-docx
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 5

In [2]:
import torch
import zipfile
from transformers import GPT2Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch import Tensor
import ast
import re
import os
from chromadb.config import Settings
from chromadb.api.models.Collection import Collection
import pandas as pd
from time import sleep
from tqdm import tqdm
import chromadb
import numpy as np
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
import gdown
from docx import Document
from google.colab import drive
import requests
import zipfile

In [3]:
client = chromadb.PersistentClient(path="/content/chroma")
collection = client.get_or_create_collection("test")

In [4]:
sentence_model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [5]:
class RecursiveCharacterTextSplitter:
    def __init__(self, chunk_size, chunk_overlap):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_text(self, text):
        # Инициализация списка частей текста
        chunks = []
        current_chunk = ""
        # Разделение текста на предложения с помощью регулярного выражения
        sentences = re.split(r'(?<=[^A-ZА-Я]\.)\s+(?=[A-ZА-Я])', text)

        for sentence in sentences:
            # Проверка, поместится ли предложение в текущую часть
            if len(current_chunk) + len(sentence.split()) <= self.chunk_size:
                current_chunk += sentence + " "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = " ".join(sentence.split()[-self.chunk_overlap:]) + " "

        # Добавление последней части, если она есть
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks


def read_text_from_file(file_path):
    if file_path.endswith('.docx'):
        doc = Document(file_path)
        return '\n'.join(paragraph.text for paragraph in doc.paragraphs)
    elif file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            return '\n'.join(reader.pages[i].extract_text() for i in range(len(reader.pages)))
    else:
        raise ValueError("Unsupported file format. Only .docx and .pdf are supported.")

def load_documents(file_paths):
    documents = []

    for file_path in file_paths:
        print(file_path)
        file_contents = read_text_from_file(file_path)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = text_splitter.split_text(file_contents)
        documents.extend([{'text': chunk, 'file_path': file_path, 'page_number': i} for i, chunk in enumerate(chunks)])

    return documents



In [6]:
def get_file_paths(directory):
    file_paths = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_name, file_extension = os.path.splitext(file_path)

            # Если расширение файла не .pdf, изменить имя файла и добавить .pdf
            if file_extension.lower() != '.pdf':
                new_file_path = file_name + '.pdf'
                os.rename(file_path, new_file_path)
                file_paths.append(new_file_path)
            else:
                file_paths.append(file_path)

    return file_paths

directory_path = '/content/pdfs'
#file_paths = get_file_paths(directory_path)
documents = load_documents(['/content/test.pdf'])


/content/test.pdf


In [7]:
def calculate_embedding(txt, model):
    embedding = model.encode(txt, normalize_embeddings=True)
    return embedding.tolist()

def fill_documents(documents):
    df = pd.DataFrame()
    df["text"] = [doc['text'] for doc in documents]
    df["file_path"] = [doc['file_path'] for doc in documents]
    df["page_number"] = [doc['page_number'] for doc in documents]
    df["text_embeddings"] = df["text"].apply(calculate_embedding, args=(sentence_model,))
    return df

df = fill_documents(documents)

In [8]:
def collectionUpsert(df):
  texts = df["text"].tolist()
  file_path = df["file_path"].tolist()
  text_embeddings = df["text_embeddings"].tolist()
  ids = [str(i) for i in range(1, len(df['text'])+1)]
  collection.upsert(
      ids=ids,
      embeddings=text_embeddings,
      metadatas=[{"source": "test", "file_path": txt} for txt in file_path],
      documents=texts
  )

  return collection


In [9]:
!zip -r /content/dataset.zip /content/chroma

  adding: content/chroma/ (stored 0%)
  adding: content/chroma/chroma.sqlite3 (deflated 98%)


In [10]:
def encodeQuestion(question, model, collection):
  embedding = model.encode(question, normalize_embeddings=True)
  embedding_list = embedding.tolist()
  result = collection.query(query_embeddings=[embedding_list])
  result_documents = []

  for distance, metadata, document in zip(
          result["distances"][0], result["metadatas"][0], result["documents"][0]
  ):
      if distance < 1:
          result_documents.append({"file_path": metadata["file_path"], "answer": document, "metric":distance})
  result_documents.sort(key=lambda x: x["metric"], reverse=False)
  result_documents = result_documents[:3]

  return result_documents

In [11]:
qa_template = """
        <SC1>Вы - банковский ассистент. Вы отвечаете на вопрос с помощью информации,вы можете использовать ее при ответе если вопрос.
        Отвечай только на то, о чем тебя спросили. Не задавай встречных вопросов.
        Не повторяй информацию, только отвечай на вопрос. Давай один точный ответ только по информации. Не повторяй вопрос
        Информация: {context}
        <SC6> Вопрос: {question}
        <extra_id_0>
        """

In [12]:
# Фунцкия нужна для правильного формирования ответа для пользователя
def remove_after_question(text):
    question_pos = text.find("Вопрос :")
    if question_pos != -1:
        return text[:question_pos]
    else:
        return text

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/FRED-T5-1.7B')
device='cuda'
model.to(device)


tokenizer_config.json:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.96G [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(50364, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50364, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
       

In [14]:
collection_result = collectionUpsert(df)
while True:
  question = input("Введите запрос: ")
  top_answer = encodeQuestion(question, sentence_model,collection_result)
  result = []
  for answer in top_answer:
    result.append(answer['answer'])
  if len(top_answer) == 0:
    qa = qa_template.format(context="", question=question)
  else:
    qa = qa_template.format(context=result, question=question)
  input_ids=torch.tensor([tokenizer.encode(qa, add_special_tokens=True)]).to(device)
  outputs=model.generate(input_ids,
                            top_p = 0.4,
                            temperature=0.2,
                            repetition_penalty =2.0,
                            min_length = 20,
                            max_length = 600,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)
  new_text = remove_after_question(tokenizer.decode(outputs[0][1:]))
  print(new_text)

Введите запрос: Расскажи все о статье 1


<extra_id_0>  Ответ: Российская Федерация - Россия есть демократическое федеративное правовое государство с республиканской формой правления.
         


KeyboardInterrupt: Interrupted by user